In [1]:
# Importar as bibliotecas necessárias para este projeto
import numpy as np
import pandas as pd
from tqdm import tqdm, tqdm_pandas

print("Libraries loaded!")

Libraries loaded!


In [2]:
try:
    train = pd.read_csv('../../data/train.csv')
except e:
    print("Error on trying read train dataset.")
finally:
    print("dataset loaded")


dataset loaded


In [3]:
# Taking a look at how many rows and columns the train dataset contains
rows = train.shape[0]
columns = train.shape[1]
print("The train dataset contains {0} rows and {1} columns".format(rows, columns))

The train dataset contains 595212 rows and 59 columns


In [ ]:
train.head()




In [ ]:
display(train.describe())


In [ ]:
%matplotlib inline

In [ ]:
series = pd.Series([train['target'].sum(), len(train.values)], index=['1', '0'], name='train')
series.plot.pie(figsize=(7, 7), autopct='%.2f', fontsize=16)

In [4]:
ind_vars = [] 
reg_vars = []
car_vars = []
calc_vars = []
rest_vars = []

bin_vars = []
cat_vars = []
num_ord_vars = []

for f in train.columns:
    if 'ind' in f:
        ind_vars.append(f)
    elif 'reg' in f:
        reg_vars.append(f)
    elif 'car' in f:
        car_vars.append(f)
    elif 'calc' in f:
        calc_vars.append(f)
    else:
        rest_vars.append(f)
        
    if 'bin' in f:
        bin_vars.append(f)
    elif 'cat' in f:
        cat_vars.append(f)
    else:
        num_ord_vars.append(f)
        
print('There are {} variables of grouping ind'.format(len(ind_vars)))
print('There are {} variables of grouping reg'.format(len(reg_vars)))
print('There are {} variables of grouping car'.format(len(car_vars)))
print('There are {} variables of grouping calc'.format(len(calc_vars)))
print('\n')
print('There are {} binary variables'.format(len(bin_vars)))
print('There are {} categorical variables'.format(len(cat_vars)))
print('There are {} ordinal/numerical variables'.format(len(num_ord_vars)))
print("\n")
print("So later on we can create dummy variables for the 14 categorical variables.")
print("The ordinal/numerical variables we can use as such and the bin variables are already binary.")

There are 18 variables of grouping ind
There are 3 variables of grouping reg
There are 16 variables of grouping car
There are 20 variables of grouping calc


There are 17 binary variables
There are 14 categorical variables
There are 28 ordinal/numerical variables


So later on we can create dummy variables for the 14 categorical variables.
The ordinal/numerical variables we can use as such and the bin variables are already binary.


In [ ]:
import missingno as msno

train_copy = train
train_copy = train_copy.replace(-1, np.NaN)

# any() applied twice to check run the isnull check across all columns.
train_copy.isnull().any().any()
msno.bar(train_copy)

In [5]:
vars_with_missing = []

for f in train.columns:
    missings = train[train[f] == -1][f].count()
    if missings > 0:
        vars_with_missing.append(f)
        missings_perc = missings/train.shape[0]
        
        print('Variable {} has {} records ({:.2%}) with missing values'.format(f, missings, missings_perc))
        
print('In total, there are {} variables with missing values'.format(len(vars_with_missing)))
    

Variable ps_ind_02_cat has 216 records (0.04%) with missing values
Variable ps_ind_04_cat has 83 records (0.01%) with missing values
Variable ps_ind_05_cat has 5809 records (0.98%) with missing values
Variable ps_reg_03 has 107772 records (18.11%) with missing values
Variable ps_car_01_cat has 107 records (0.02%) with missing values
Variable ps_car_02_cat has 5 records (0.00%) with missing values
Variable ps_car_03_cat has 411231 records (69.09%) with missing values
Variable ps_car_05_cat has 266551 records (44.78%) with missing values
Variable ps_car_07_cat has 11489 records (1.93%) with missing values
Variable ps_car_09_cat has 569 records (0.10%) with missing values
Variable ps_car_11 has 5 records (0.00%) with missing values
Variable ps_car_12 has 1 records (0.00%) with missing values
Variable ps_car_14 has 42620 records (7.16%) with missing values
In total, there are 13 variables with missing values


ps_car_03_cat and ps_car_05_cat have a large proportion of records with missing values. Remove these variables.

For the other categorical variables with missing values, we can leave the missing value -1 as such.

ps_reg_03 (continuous) has missing values for 18% of all records. Replace by the mean.

ps_car_11 (ordinal) has only 5 records with misisng values. Replace by the mode.

ps_car_12 (continuous) has only 1 records with missing value. Replace by the mean.

ps_car_14(continuous) has missing values for 7% of all records. Replace by the mean.

In [ ]:
# ps_car_03_cat and ps_car_05_cat have a large proportion of records with missing values. Remove these variables.
#train.drop(['ps_car_03_cat', 'ps_car_05_cat'], inplace=True, axis=1)
#cat_vars.remove('ps_car_03_cat')
#cat_vars.remove('ps_car_05_cat')

#print("removing features done")

In [6]:
from sklearn.preprocessing import Imputer

# Imputing with the mean or mode
mean_imp = Imputer(missing_values=-1, strategy='mean', axis=0)
mode_imp = Imputer(missing_values=-1, strategy='most_frequent', axis=0)
train['ps_reg_03'] = mean_imp.fit_transform(train[['ps_reg_03']]).ravel()
train['ps_car_12'] = mean_imp.fit_transform(train[['ps_car_12']]).ravel()
train['ps_car_14'] = mean_imp.fit_transform(train[['ps_car_14']]).ravel()
train['ps_car_11'] = mode_imp.fit_transform(train[['ps_car_11']]).ravel()

print("Imputing done")

Imputing done


In [ ]:
#Only categorical variable ps_car_11_cat has a bit more distinct values, although it is still reasonable. 
#To avoid having many dummy variables later on, we could replace the values 
#in this variable by the supervised ratio. Other strategies to transform this
#variable are explained in an article on KDNuggets. 
#As a result this variable can then be used as a continuous variable.


for f in cat_vars:
    dist_values = train[f].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(f, dist_values))

In [7]:
train.to_csv('../../data/train_prepared.csv',index=False)

** @@@@@@@@@@@@@@@@@@@@@@@@@@@ **

In [ ]:
# Produza uma matriz de dispersão para cada um dos pares de atributos dos dados
#pd.scatter_matrix(data, alpha = 0.3, figsize = (18,12), diagonal = 'kde');
train_target = train_data.pop('target')
train_ids = train_data.pop('id')
train_data


In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls

py.init_notebook_mode(connected=True)
# Formatação mais bonita para os notebooks

# Nullity or missing values by columns
#msno.matrix(df=train_copy.iloc[:,0:30], figsize= (20, 14), color=(0.42, 0.1, 0.05))

#msno.matrix(df=train_copy.iloc[:,30:57], figsize=(20, 14), color=(0.42, 0.1, 0.05))


In [ ]:
colwithnan = train_copy.columns[train_copy.isnull().any()].tolist()

print("Just a reminder this dataset has %s Rows. \n" % (train_copy.shape[0]))
for col in colwithnan:
    print("Column: %s has %s NaN" % (col, train_copy[col].isnull().sum()))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=150, max_depth=8, min_samples_leaf=4, max_features=0.2, n_jobs=-1, random_state=0)
rf.fit(train_data, train_target)
features = train_data.columns.values

print("----- Training Done -----")

In [ ]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

for f in range(train_data.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))


In [ ]:
# Scatter plot 
trace = go.Scatter(
    y = rf.feature_importances_,
    x = features,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 13,
        #size= rf.feature_importances_,
        #color = np.random.randn(500), #set color equal to a variable
        color = rf.feature_importances_,
        colorscale='Portland',
        showscale=True
    ),
    text = features
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
     xaxis= dict(
         ticklen= 5,
         showgrid=False,
        zeroline=False,
        showline=False
     ),
    yaxis=dict(
        title= 'Feature Importance',
        showgrid=False,
        zeroline=False,
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

In [ ]:
x, y = (list(x) for x in zip(*sorted(zip(rf.feature_importances_, features), 
                                                            reverse = False)))
trace2 = go.Bar(
    x=x ,
    y=y,
    marker=dict(
        color=x,
        colorscale = 'Viridis',
        reversescale = True
    ),
    name='Random Forest Feature importance',
    orientation='h',
)

layout = dict(
    title='Barplot of Feature importances',
     width = 900, height = 2000,
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
#         domain=[0, 0.85],
    ))

fig1 = go.Figure(data=[trace2])
fig1['layout'].update(layout)
py.iplot(fig1, filename='plots')

In [ ]:
important_feature = []
for f in range(28):
    important_feature.append(indices[f])
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
print()
print(important_feature)

## Machine Learning

In [ ]:
test_data = pd.read_csv('../../data/test.csv')
# Êxito
print("O conjunto de dados de tem {} pontos com {} variáveis em cada.".format(*test_data.shape))


test_ids = test_data.pop('id')


In [ ]:
def gini(actual, pred, cmpcol = 0, sortcol = 1):
    assert( len(actual) == len(pred) )
    all = np.asarray(np.c_[ actual, pred, np.arange(len(actual)) ], dtype=np.float)
    all = all[ np.lexsort((all[:,2], -1*all[:,1])) ]
    totalLosses = all[:,0].sum()
    giniSum = all[:,0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)
 
def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return [('gini', gini_score)]

In [ ]:
one_enc = OneHotEncoder()

one_enc.fit(train_data[vals_types['cat']])

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(train_data, train_target, test_size=0.2, random_state=4242)
print('Train samples: {} Validation samples: {}'.format(len(x_train), len(x_valid)))

In [ ]:
d_train = xgb.DMatrix(x_train, y_train)
d_valid = xgb.DMatrix(x_valid, y_valid)
d_test = xgb.DMatrix(test_data)

In [ ]:
# Set xgboost parameters
params = {}
params['objective'] = 'binary:logistic'
params['eta'] = 0.02
params['silent'] = True
params['max_depth'] = 4
params['subsample'] = 0.9
params['colsample_bytree'] = 0.9
params['eval_metric'] = 'auc'
params['silent'] = True

In [ ]:
# This is the data xgboost will test on after eachboosting round
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

In [ ]:
# Train the model! We pass in a max of 10,000 rounds (with early stopping after 100)
# and the custom metric (maximize=True tells xgb that higher metric is better)

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
import gc


print('loading files...')
train = pd.read_csv('../../data/train.csv', na_values=-1) 
test = pd.read_csv('../../data/test.csv', na_values=-1)
col_to_drop = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(col_to_drop, axis=1)  
test = test.drop(col_to_drop, axis=1)  

for c in train.select_dtypes(include=['float64']).columns:
    train[c]=train[c].astype(np.float32)
    test[c]=test[c].astype(np.float32)
for c in train.select_dtypes(include=['int64']).columns[2:]:
    train[c]=train[c].astype(np.int8)
    test[c]=test[c].astype(np.int8)    

print(train.shape, test.shape)
    
# xgb
params = {'eta': 0.02, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 0.9, 
          'objective': 'binary:logistic', 'eval_metric': 'auc', 'silent': True}

X = train.drop(['id', 'target'], axis=1)
features = X.columns
X = X.values
y = train['target'].values
sub=test['id'].to_frame()
sub['target']=0

nrounds=2000  # need to change to 2000
kfold = 5  # need to change to 5
skf = StratifiedKFold(n_splits=kfold, random_state=0)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(' xgb kfold: {}  of  {} : '.format(i+1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    d_train = xgb.DMatrix(X_train, y_train) 
    d_valid = xgb.DMatrix(X_valid, y_valid) 
    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    xgb_model = xgb.train(params, d_train, nrounds, watchlist, early_stopping_rounds=100, 
                          feval=gini_xgb, maximize=True, verbose_eval=100)
    sub['target'] += xgb_model.predict(xgb.DMatrix(test[features].values), 
                        ntree_limit=xgb_model.best_ntree_limit+50) / (2*kfold)
    
sub.to_csv('submission.csv', index=False, float_format='%.5f')
gc.collect()
sub.head(2)



In [ ]:
p_test = xgb_model.predict(d_test)

In [ ]:
xgb_model.best_score

#0.258604

In [ ]:
# Create a submission file
sub = pd.DataFrame()
sub['id'] = test_ids
sub['target'] = p_test
sub.to_csv('submission.csv', index=False,float_format='%.5f')

print(sub.head())

https://www.kaggle.com/bertcarremans/data-preparation-exploration

https://www.kaggle.com/arthurtok/interactive-porto-insights-a-plot-ly-tutorial

https://www.kaggle.com/tezdhar/faster-gini-calculation

https://github.com/snovik75/porto_seguro/blob/master/notebooks/model.ipynb

https://github.com/maksimovkonstantin/KagglePortoSeguro/blob/master/Simple%20model%20creation.ipynb

https://github.com/search?p=3&q=porto-seguro&type=Repositories&utf8=%E2%9C%93

https://www.kaggle.com/batzner/gini-coefficient-an-intuitive-explanation


https://www.kaggle.com/jeru666/easy-to-fork-porto-seguro